In [1]:
# Import packages
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
from tqdm.notebook import tqdm_notebook
# Create the folder for saving files (if it doesn't exist)
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['.ipynb_checkpoints',
 'basics.csv',
 'movies-erd.png',
 'ratings.csv',
 'test',
 'title-akas-us-only.csv',
 'title.basics.tsv.gz',
 'title.ratings.tsv.gz',
 'tmdb_api_results_2010.json']

In [2]:
# load login credentials
with open(".secret/tmdb_api.json", "r") as f:
    login = json.load(f)
    
## Display the keys of the loaded dict
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
tmdb.API_KEY =  login['api-key']

In [4]:
def write_json(new_data, filename): 
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""  
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [5]:
def get_movie_with_rating(movie_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(movie_id)
        # save the .info .releases dictionaries
    movie_info = movie.info()
    releases = movie.releases()
        # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
            movie_info['certification'] = c['certification']
    return movie_info

In [6]:
basics = pd.read_csv('Data/basics.csv')

basics

movie = tmdb.Movies('tt0848228')

movie_info = movie.info()

movie_info

{'adult': False,
 'backdrop_path': '/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 96.383,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path'

In [7]:
get_movie_with_rating('tt0332280')

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 54.796,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/mevhneWSqbjU22D1MXNd4H9x0r0.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [8]:
# Define a variable with the year to Extract from the API
YEAR = 2010

In [9]:
# define empty list for any movies that cause an error

errors = []

In [10]:
# define a JSON file to store year results
JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'

In [11]:
# determine if JSON_FILE already exists
file_exists = os.path.isfile(JSON_FILE)

In [12]:
# if-else so that if it does not exist it will be created

if file_exists == False:
    # Print a message indicating the file is being created 
    print(f"Creating {JSON_FILE} for API results for {YEAR}.")
    # save an empty dict with just "imdb_id" to the new json file.
    with open(JSON_FILE,'w') as f:
        json.dump([{'imdb_id':0}],f)
        
else:
    print(f"The file {JSON_FILE} already exists.")

The file Data/tmdb_api_results_2010.json already exists.


In [16]:
# create new dataframe, df, with filter for selected YEAR
df = basics.loc[basics['startYear'] == YEAR].copy()

# save list of movie ids to separate variable movie_ids
movie_ids = df['tconst']

1138    tt0230212
3806    tt0312305
4258    tt0326965
4436    tt0331312
6670    tt0393049
Name: tconst, dtype: object

### Check previous results and create the final list of movie_ids_to_get

In [17]:
# load in existing API results into previous_df dataframe as safeguard
previous_df = pd.read_json(JSON_FILE)

In [18]:
# check for existing API results
previous_df

,imdb_id
0,0


### Check for and filter out movie IDs that already exist


In [19]:
# filter out any ids already in JSON_FILE
movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]

### Loop through Movie IDs

In [ ]:
# Loop through movie_ids_to_get with a tqdm progress bar
for movie_id in tqdm_notebook(movie_ids_to_get, f"Movies from {YEAR}"):
    try:
    # Retrieve then data for the movie id
        temp = get_movie_with_rating(movie_id)  
    # Append/extend results to existing file using a pre-made function
        write_json(temp,JSON_FILE)
    # Short 20 ms sleep to prevent overwhelming server
        time.sleep(0.02)
    
    except Exception as e:
        errors.append([movie_id, e])

Movies from 2010:   0%|          | 0/3862 [00:00<?, ?it/s]

In [ ]:
print(f"- Total errors: {len(errors)}")

In [ ]:
print(f"- Total errors: {len(errors)}")
